In [ ]:
import geemap
import ee
import numpy as np
import pandas as pd

In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()

## Run javascript snippets

As first starting point, translate copy-pasted Javascript code to Python and execute locally.

In [ ]:
js_snippet = """

// Example script to load and visualize ERA5 climate reanalysis parameters in
// Google Earth Engine

// Daily mean 2m air temperature
var era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY')
                   .select('mean_2m_air_temperature')
                   .filter(ee.Filter.date('2019-07-01', '2019-07-31'));

// Visualization palette for temperature (mean, min and max) and 2m dewpoint
// temperature
var vis2mt = {
  min: 250,
  max: 320,
  palette: [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
};

Map.addLayer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')), vis2mt,
    'Daily mean 2m air temperature');

Map.setCenter(21.2, 22.2, 2);

"""

In [ ]:
geemap.js_snippet_to_py(js_snippet, import_ee=False, import_geemap=False)

In [ ]:

# Example script to load and visualize ERA5 climate reanalysis parameters in
# Google Earth Engine

# Daily mean 2m air temperature
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
                   .select('mean_2m_air_temperature') \
                   .filter(ee.Filter.date('2019-07-01', '2019-07-31'))

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

Map.addLayer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')), vis2mt,
    'Daily mean 2m air temperature')

Map.setCenter(21.2, 22.2, 2)
Map


## Inspect temperature data

Let's further inspect the temperature data image collection

In [ ]:
def extract_single_img(collection, idx=0):
    
    single_img = ee.Image(collection.toList(collection.size()).get(idx))
    
    return single_img

In [ ]:
def extract_img_metadata(img, list_of_properties, img_id=0):
    
    props = {}
    props[img_id] = {k: img['properties'][k] for k in list_of_properties}
    
    return props

In [ ]:
def extract_img_collection_metadata(this_collection, list_of_properties):
    
    n_imgs = len(this_collection.getInfo()['features'])
    img_features = this_collection.getInfo()['features']
    
    all_img_props = []
    
    for idx in np.arange(0, n_imgs):
        
        this_img_metadata = extract_img_metadata(img_features[idx], list_of_properties, idx)
        all_img_props.append(pd.DataFrame().from_dict(this_img_metadata).T)
        
    return pd.concat(all_img_props, axis=0)

In [ ]:
# Daily mean 2m air temperature
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
                   .select('mean_2m_air_temperature') \
                   .filter(ee.Filter.date('2019-07-01', '2019-07-31'))

Get some metadata for a single image:

In [ ]:
single_tmp_img = extract_single_img(era5_2mt)
list_of_properties = ['year', 'month', 'day']

extract_img_metadata(single_tmp_img.getInfo(), list_of_properties)

Extract date / time info for all images of the collection

In [ ]:
extract_img_collection_metadata(era5_2mt, list_of_properties).head(7)

## Create gif

Set a region in the following map. For this region we will visualize temperature data over time as a `gif`

In [ ]:
Map = geemap.Map()
Map.setCenter(21.2, 22.2, 2)
Map

In [ ]:
# color gradient for Celsius
celsius_color_grad = {
  'min': -40,
  'max': 35,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

In [ ]:
def func_img_viz(img):
    return img.subtract(273.5).visualize(**celsius_color_grad)

In [ ]:
# Define GIF visualization arguments.
gifParams = {
  'region': Map.draw_last_feature.bounds(),
  'dimensions': 600,
  'framesPerSecond': 3,
  'format': 'gif'
}

### Single year, one image per month

In [ ]:
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
                   .select('mean_2m_air_temperature') \
                   .filter(ee.Filter.date('2019-01-01', '2019-12-31')) \
                   .filterMetadata('day', 'equals', 1)

In [ ]:
months = extract_img_collection_metadata(era5_2mt, list_of_properties)['month']

In [ ]:
viz_img_coll = era5_2mt.map(func_img_viz)

In [ ]:
geemap.download_ee_video(viz_img_coll, gifParams, '/home/chris/Downloads/geemap.gif')

In [ ]:
geemap.add_text_to_gif('/home/chris/Downloads/geemap.gif', 
                       '/home/chris/Downloads/geemap_annot.gif',
                       text_sequence=months,
                       duration=600
              )

In [ ]:
geemap.show_image('/home/chris/Downloads/geemap_annot.gif')

### One image per year

In [ ]:
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
                   .select('mean_2m_air_temperature') \
                   .filter(ee.Filter.And(ee.Filter.eq('day', 1), ee.Filter.inList('month', ee.List([6]))))

In [ ]:
years = extract_img_collection_metadata(era5_2mt, list_of_properties)['year']

In [ ]:
viz_img_coll = era5_2mt.map(func_img_viz)

In [ ]:
geemap.download_ee_video(viz_img_coll, gifParams, '/home/chris/Downloads/geemap.gif')

In [ ]:
geemap.show_image('/home/chris/Downloads/geemap.gif')

In [ ]:
geemap.add_text_to_gif('/home/chris/Downloads/geemap.gif', 
                       '/home/chris/Downloads/geemap_annot.gif',
                       text_sequence=years,
                       duration=200
              )

In [ ]:
geemap.show_image('/home/chris/Downloads/geemap_annot.gif')

In [ ]:
width = 250
height = 30
palette = ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
labels = [-40, 35]
cbar = geemap.create_colorbar(width=width, height=height, palette=palette, vertical=True, labels=labels)

In [ ]:
geemap.show_image(cbar)

In [ ]:
geemap.add_image_to_gif(in_gif='/home/chris/Downloads/geemap_annot.gif', 
                        out_gif='/home/chris/Downloads/geemap_annot.gif',
                        in_image=cbar,
                        xy=(20, 80),
                        image_size=(80, 80)
                       )

In [ ]:
geemap.show_image('/home/chris/Downloads/geemap_annot.gif')

## Landsat

In [ ]:
def extract_img_metadata(img):
    
    list_of_properties = ['WRS_PATH', 'WRS_ROW', 'DATE_ACQUIRED', 
                          'SCENE_CENTER_TIME', 'CLOUD_COVER', 'CLOUD_COVER_LAND'
                         ]
    img_id = img['properties']['LANDSAT_PRODUCT_ID']
    
    props = {}
    props[img_id] = {k: img['properties'][k] for k in list_of_properties}
    
    return props